In [2]:
from fastapi import FastAPI
import pandas as pd
import pickle

app = FastAPI()

# Define a root `/` endpoint
@app.get('/')
def index():
    return {'ok': True}

@app.get("/predict")
def predict(
        Location: object,
        Age: int,
        Gender: object,
        Product_Name: object,
        Selling_Price: float,
        Interaction_type: object,
        Product_Description: object,
    ):
    """
    Make a single course prediction.
    Assumes `pickup_datetime` is provided as a string by the user in "%Y-%m-%d %H:%M:%S" format
    Assumes `pickup_datetime` implicitly refers to the "US/Eastern" timezone (as any user in New York City would naturally write)
    """
    data_dict = {
        'Location': [Location],
        'Age': [Age],
        'Gender': [Gender],
        'Product_Name': [Product_Name],
        'Selling_Price': [Selling_Price],
        'Interaction_type': [Interaction_type],
    }

    X_pred = pd.DataFrame(data_dict, index=[0])

    with open('models/preprocessor.pkl', 'rb') as f:
        preprocessor = pickle.load(f)

    preprocessed_data = pd.DataFrame(preprocessor.transform(X_pred))

    with open('models/harp_model.pkl', 'rb') as f:
        model = pickle.load(f)

    # Make prediction using the loaded model
    prediction = model.predict(preprocessed_data)

    # Return the prediction result
    return {'result': prediction[0]}
